In [1]:
from MixedAutoencoder import Mixer, MixedAutoencoder
import MixedAutoencoder
from DataCleaning import *
import DataCleaning
import pandas as pd
import random
import matplotlib.pyplot as plt
import numpy as np
seed = 42
MixedAutoencoder.setRandom(seed)
DataCleaning.setRandom(seed)
base_path = "."
mixer = Mixer(base_path)

In [2]:

num_sets = 4
latent_dim = 7
model_shape = []
base_path = "."
label = f'demo_{num_sets}_{latent_dim}_[{"_".join([str(s) for s in model_shape])}]'
demo_size = 20
demo_shape = []


In [3]:
base_key_list = [f'set{str(i)}' for i in range(1, num_sets+1)]
demo_key_list = ['demo']

data = pd.read_csv(f'{base_path}/data/16PF/data.csv', sep="\t")
data = clean_data(data)
data = data.sample(frac=1)
base_column_keys = split_strat(data,num_sets, base_key_list)
demo_column_keys = {demo_key_list[0]: split_n_strat(data, demo_size)}
split_data = split(data, base_column_keys)
train, test = make_train_test(split_data, 0.8)
input_dims = get_input_dims(train)

20


In [4]:
model_shapes = {k: model_shape for k in base_key_list}
autoencoder_set = mixer.make_new(model_shapes, latent_dim, input_dims)
autoencoder_set.show_total_binary_accuracy(test)
settings = {
    "training": [["$all", "$all", True, True]]#[[[k1], [k2 for k2 in keys if k1 != k2], True, True] for k1 in keys],
    #"encoder_proximity_training": [["$all", True]],
    #"plot": [True, 3, [0, 1, 2]]
}
autoencoder_set.train_set(train, 20, autoencoder_set.make_train_config(settings = settings), batch_size = 64, verbose=True)
autoencoder_set.show_total_binary_accuracy(test)

Binary accuracy: 0.49382968122050225
epoch 0
442/442 [==============================] - 4s 8ms/step - loss: 1.1438 - val_loss: 1.2548 - accuracy: 0.2699
epoch 1
442/442 [==============================] - 3s 5ms/step - loss: 0.9835 - val_loss: 1.1096 - accuracy: 0.3304
epoch 2
442/442 [==============================] - 3s 5ms/step - loss: 0.9127 - val_loss: 1.0332 - accuracy: 0.3555
epoch 3
442/442 [==============================] - 3s 5ms/step - loss: 0.8614 - val_loss: 0.9894 - accuracy: 0.3701
epoch 4
442/442 [==============================] - 3s 5ms/step - loss: 0.8286 - val_loss: 0.9672 - accuracy: 0.3808
epoch 5
442/442 [==============================] - 3s 5ms/step - loss: 0.8102 - val_loss: 0.9515 - accuracy: 0.3874
epoch 6
442/442 [==============================] - 3s 5ms/step - loss: 0.7990 - val_loss: 0.9388 - accuracy: 0.3928
epoch 7
442/442 [==============================] - 4s 5ms/step - loss: 0.7912 - val_loss: 0.9288 - accuracy: 0.3973
epoch 8
442/442 [==================

In [5]:
demo_split_data = split(data, demo_column_keys)
dtrain, dtest = make_train_test(demo_split_data, 0.8)
demo_input_dims = get_input_dims(dtrain)




demo_shapes = {k: demo_shape for k in demo_key_list}
autoencoder_set = mixer.add_new(autoencoder_set, demo_shapes, demo_input_dims)

settings = {
    "training": [[demo_key_list, "$all", True, False], ["$all", demo_key_list, False, True]]#[[[k1], [k2 for k2 in keys if k1 != k2], True, True] for k1 in keys],
    #"encoder_proximity_training": [["$all", True]],
    #"plot": [True, 3, [0, 1, 2]]
}
autoencoder_set.train_set(train | dtrain, 20, autoencoder_set.make_train_config(settings = settings), batch_size = 64, verbose=True)
autoencoder_set.show_binary_accuracy(demo_key_list, test | dtest)


epoch 0
442/442 [==============================] - 2s 5ms/step - loss: 1.4708 - val_loss: 1.2545 - accuracy: 0.3084
epoch 1
442/442 [==============================] - 2s 4ms/step - loss: 1.2022 - val_loss: 1.0653 - accuracy: 0.3505
epoch 2
442/442 [==============================] - 2s 4ms/step - loss: 1.0431 - val_loss: 0.9349 - accuracy: 0.3914
epoch 3
442/442 [==============================] - 3s 4ms/step - loss: 0.9587 - val_loss: 0.8645 - accuracy: 0.4067
epoch 4
442/442 [==============================] - 2s 4ms/step - loss: 0.9206 - val_loss: 0.8291 - accuracy: 0.4166
epoch 5
442/442 [==============================] - 2s 4ms/step - loss: 0.9032 - val_loss: 0.8129 - accuracy: 0.4277
epoch 6
442/442 [==============================] - 2s 4ms/step - loss: 0.8944 - val_loss: 0.8052 - accuracy: 0.4328
epoch 7
442/442 [==============================] - 2s 4ms/step - loss: 0.8900 - val_loss: 0.8013 - accuracy: 0.4364
epoch 8
442/442 [==============================] - 2s 4ms/step - loss: 0

In [6]:

mixer.save_to_label(autoencoder_set, extra = {"columns" : base_column_keys|demo_column_keys}, label = label)

INFO:tensorflow:Assets written to: ./Models/model_demo_4_7_[]/encoder_set1\assets
INFO:tensorflow:Assets written to: ./Models/model_demo_4_7_[]/decoder_set1\assets
INFO:tensorflow:Assets written to: ./Models/model_demo_4_7_[]/encoder_set2\assets
INFO:tensorflow:Assets written to: ./Models/model_demo_4_7_[]/decoder_set2\assets
INFO:tensorflow:Assets written to: ./Models/model_demo_4_7_[]/encoder_set3\assets
INFO:tensorflow:Assets written to: ./Models/model_demo_4_7_[]/decoder_set3\assets
INFO:tensorflow:Assets written to: ./Models/model_demo_4_7_[]/encoder_set4\assets
INFO:tensorflow:Assets written to: ./Models/model_demo_4_7_[]/decoder_set4\assets
INFO:tensorflow:Assets written to: ./Models/model_demo_4_7_[]/encoder_demo\assets
INFO:tensorflow:Assets written to: ./Models/model_demo_4_7_[]/decoder_demo\assets
